In [3]:
# import necessary libraries
import time
import datetime
import numpy as np
import cvxpy as cp

import os
# os.chdir("Git/Synth")

from utils.synth import *
from utils.sim_ar import *
from utils.competitors import *
from cvxopt import matrix, solvers
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)

In [ ]:
import numpy as np

J=100
# Example vector
vector = np.ones(J)/J

# Calculate the L2 norm (Euclidean norm)
l2_norm = np.linalg.norm(vector)
print("L2 Norm:", l2_norm)

# Calculate the L-inf norm (Infinity norm)
l_inf_norm = np.linalg.norm(vector, np.inf)
print("L-inf Norm:", l_inf_norm)

In [24]:
T0 = 100
T1 = 5
J = 30
rho1, rho2 = 0, 0
gamma = 0
trt_effect = 5
sig2 = 1
sig = np.sqrt(sig2)
DGP = 2
fixed_alpha = None
fixed_lam = None
pollution = None
scale = False
intercept = False

In [64]:
T01 = T0 + T1
F1 = np.random.normal(scale=1, size=T01)
F2 = np.random.normal(scale=2, size=T01)

F1 += 4
F2 += 4

lambda1 = np.arange(1, J + 1) / J
lambda2 = lambda1.copy()

q2, r2 = divmod(J, 2)
q3, r3 = divmod(J, 3)
w1 = np.ones(J) / J
w2 = np.concatenate(([-2/J]*(q2), [2/J]*(q2+r2))) + np.random.uniform(-1/(J**2), 1/(J**2), J)
np.random.shuffle(w2)
w3 = np.concatenate(([-2/J]*(q3), [2/J]*(q3), np.zeros(q3+r3)))
np.random.shuffle(w3)
# w6 = np.concatenate((np.zeros(J-4), [-2/3, -2/3, 2/3, 2/3]))
# np.random.shuffle(w6)
# w5 = np.random.uniform(-4/J, 4/J, J)
w4 = np.random.beta(0.2, 0.2, J)
w4 = (w4 - 0.5) * 3/J
w = np.outer(w1, (DGP == 1)) + np.outer(w2, (DGP == 2)) + \
    np.outer(w3, (DGP == 3)) + np.outer(w4, (DGP == 4))
y = np.kron(np.ones(T01), lambda1) + np.kron(F1, np.ones(J)) + \
    np.kron(F2, lambda2)

eps = np.empty((T01, J))
for j in range(J):
    eps[:, j] = generate_ARMA_series(T01, rho1, gamma)

Y0 = y.reshape(T01, J) + eps
u = generate_ARMA_series(T01, rho2, gamma)
# if DGP == 6:
#     Y1 = np.zeros((T01, 1))
#     Y1[:T0] = Y0[:T0] @ w
#     Y1[T0:] = Y0[T0:] @ w * 0.95
# else: 

# if DGP <= 3:
#     Y1 = Y0 @ w + 0.1 * u.reshape(-1, 1)
# else:
#     Y1 = Y0 @ w + 1.0 * u.reshape(-1, 1)
Y1 = Y0 @ w + sig * u.reshape(-1, 1)
Y1 = Y1.flatten()
Y1[T0:] += trt_effect

## separate into pre-treatment and post-treatment
Y0_pre = Y0[:T0,:]
Y0_post = Y0[T0:,:]
Y1_pre = Y1[:T0]
Y1_post = Y1[T0:]

## add some contamination to the data
if pollution is not None:
    Y0_post = pollute(Y0_post, w, pollution)

In [65]:
## competitors
# w_did = np.nan
# w_l1asso = l1asso(Y1_pre, Y0_pre, 1)
w_sc = sc(Y1_pre, Y0_pre)    

## new loo
# idxs = np.random.choice(T0, T0, replace=False)
# idxs1, idxs2 = idxs[:T0//2], idxs[T0//2:]
idxs1 = np.arange(T0)
idxs2 = idxs1.copy()
# alpha_sum_l2, lam_sum_l2 = loo_selector_parallel(Y0_post, method='sum-l2')
# alpha_sum_inf, lam_sum_inf = loo_selector_parallel(Y0_post, method='sum-inf')
alpha_inf, lam_inf = loo_selector_parallel(Y1_pre[idxs1], Y0_pre[idxs1], method='inf', 
                                           fixed_alpha=fixed_alpha, fixed_lam=fixed_lam, 
                                           scale=scale, intercept=intercept)
alpha_l1, lam_l1 = loo_selector_parallel(Y1_pre[idxs1], Y0_pre[idxs1], method='l1', 
                                         fixed_alpha=fixed_alpha, fixed_lam=fixed_lam, 
                                           scale=scale, intercept=intercept)
alpha_l2, lam_l2 = loo_selector_parallel(Y1_pre[idxs1], Y0_pre[idxs1], method='l2', 
                                         fixed_alpha=fixed_alpha, fixed_lam=fixed_lam, 
                                           scale=scale, intercept=intercept)
alpha_l1_inf, lam_l1_inf = loo_selector_parallel(Y1_pre[idxs1], Y0_pre[idxs1], method='l1-inf', 
                                                 fixed_alpha=fixed_alpha, fixed_lam=fixed_lam, 
                                           scale=scale, intercept=intercept)
# alpha_l1_inf = 0.0
alpha_l1_l2, lam_l1_l2 = loo_selector_parallel(Y1_pre[idxs1], Y0_pre[idxs1], method='l1-l2', 
                                               fixed_alpha=fixed_alpha, fixed_lam=fixed_lam, 
                                           scale=scale, intercept=intercept)

# w_sum_l2 = our(Y1_pre, Y0_pre, alpha_sum_l2, lam_sum_l2, 'sum-l2')
# w_sum_inf = our(Y1_pre, Y0_pre, alpha_sum_inf, lam_sum_inf, 'sum-inf')
w_inf = our(Y1_pre[idxs2], Y0_pre[idxs2], alpha_inf, lam_inf, 'inf', 
            scale=scale, intercept=intercept)
w_l1 = our(Y1_pre[idxs2], Y0_pre[idxs2], alpha_l1, lam_l1, 'l1',  
           scale=scale, intercept=intercept)
w_l2 = our(Y1_pre[idxs2], Y0_pre[idxs2], alpha_l2, lam_l2, 'l2', 
           scale=scale, intercept=intercept)
w_l1_inf = our(Y1_pre[idxs2], Y0_pre[idxs2], alpha_l1_inf, lam_l1_inf, 'l1-inf', 
               scale=scale, intercept=intercept)
w_l1_l2 = our(Y1_pre[idxs2], Y0_pre[idxs2], alpha_l1_l2, lam_l1_l2, 'l1-l2',
              scale=scale, intercept=intercept)

# tau_did = did(Y1, Y0)[T0:]
if intercept:
    Y0_post_plus = np.hstack([np.ones((Y1_post.shape[0], 1)), Y0_post])
else:
    Y0_post_plus = Y0_post
# tau_l1asso = Y1_post - Y0_post_plus @ w_l1asso

tau_oracle = Y1_post - Y0_post @ w.flatten()
tau_sc = Y1_post - Y0_post @ w_sc
# tau_sum_l2 = Y1_post - Y0_post_plus @ w_sum_l2
# tau_sum_inf = Y1_post - Y0_post_plus @ w_sum_inf
tau_l1 = Y1_post - Y0_post_plus @ w_l1
tau_l2 = Y1_post - Y0_post_plus @ w_l2
tau_inf = Y1_post - Y0_post_plus @ w_inf
tau_l1_l2 = Y1_post - Y0_post_plus @ w_l1_l2
tau_l1_inf = Y1_post - Y0_post_plus @ w_l1_inf

In [ ]:
Tau = np.array([tau_sc, tau_l2, tau_l1, tau_l1_l2, tau_inf, tau_l1_inf, tau_oracle])
print(np.mean(np.square(Tau - trt_effect), 1))

In [67]:
# print(w_sc - w.flatten())

In [ ]:
print(w_l1_l2 - w.flatten())

In [ ]:
print(w_l1_inf - w.flatten())

In [ ]:
w.flatten()

In [ ]:
w_l2

In [ ]:
w_inf